# 🚀 RLCS Match-Level Outcome Prediction
In this notebook, we process wide-format RLCS replay frame data to create match-level features based on player stats, and use them to predict match outcomes.

## 1️⃣ Convert RLCS 2022 .parquet Files to .csv

In [ ]:
import pandas as pd

base_path = "C:/Users/Brandon/Desktop/School/ITCS 3156 - Intro to ML/FinalProject/RLCS_Outcome_Predictions/rlcs_files/"

frames = pd.read_parquet(f"{base_path}/2022_frames.parquet")
games = pd.read_parquet(f"{base_path}/2022_games.parquet")

frames.to_csv("frames.csv", index=False)
games.to_csv("games.csv", index=False)

print("✅ Parquet files converted to CSV")

## 2️⃣ Reload CSV Files

In [ ]:
frames = pd.read_csv("frames.csv")
games = pd.read_csv("games.csv")
games = games.rename(columns={"id": "game_id"})
print("✅ Reloaded and standardized CSVs")

✅ Reloaded and standardized CSVs


## 3️⃣ Build Team-Level Match Features

In [8]:
# Group by game_num to represent individual matches
match_groups = frames.groupby("game_num")

# New match level rows
match_level_rows = []

for game_num, group in match_groups:
    row = {"game_num": game_num}

    # Team 0: players p0, p1, p2
    team0_cols = [col for col in group.columns if col.startswith(("p0_", "p1_", "p2_"))]
    row.update(group[team0_cols].mean().add_prefix("team0_"))

    # Team 1: players p3, p4, p5
    team1_cols = [col for col in group.columns if col.startswith(("p3_", "p4_", "p5_"))]
    row.update(group[team1_cols].mean().add_prefix("team1_"))

    match_level_rows.append(row)

match_df = pd.DataFrame(match_level_rows)
print("✅ Match-level features created. Shape:", match_df.shape)

print(games.columns.tolist())

✅ Match-level features created. Shape: (3680, 85)
['game_num', 'game_id', 'filename', 'time', 'region', 'map', 'series_game_num', 'team0_name', 'team1_name', 'p0_name', 'p1_name', 'p2_name', 'p3_name', 'p4_name', 'p5_name']
